In [2]:
!pip install ipynb

In [3]:
! pip install -U tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 3.6 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.3.0
    Uninstalling tensorflow-datasets-4.3.0:
      Successfully uninstalled tensorflow-datasets-4.3.0


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [5]:
import sys
sys.path.insert(0,"../usr/lib")

In [6]:
import datetime
import gc
from pathlib import Path

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

In [7]:
temp_dir = Path("/kaggle/temp")
temp_dir.mkdir()

checkpoint_dir = Path("/kaggle/temp/checkpoint")
backup_dir = Path("/kaggle/temp/backup")
checkpoint_dir.mkdir()
backup_dir.mkdir()

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

del gpus

# Loading builders for datasets

In [8]:
RECIPES5K_DIR = "../input/foodnet/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "../input/foodnet/tfrecord/food101/1.0.1"
NUTRITION5K_DIR = "../input/foodnet/tfrecord/nutrition5k/1.0.0"


recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

In [9]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [10]:
nutrition5k_builder.info.splits["train"].num_examples

271407

In [11]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [12]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training category, ingredients and nutrients

In [13]:
# FULL DATASET
# (
#     recipes5k_food101_train,
#     recipes5k_food101_validation,
#     recipes5k_food101_test,
# ) = recipes5k_food101_builder.as_dataset(
#     split=["train[:70%]", "train[70%:95%]", "train[95%:]"], shuffle_files=True
# )

# # Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
# (
#     nutrition5k_category_train,
#     nutrition5k_category_val,
#     nutrition5k_category_test,
# ) = nutrition5k_builder.as_dataset(
#     split=["train[:700]", "train[700:950]", "train[950:1000]"], shuffle_files=True
# )

# train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
# validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)
# test_dataset = recipes5k_food101_test.concatenate(nutrition5k_category_test)

In [14]:
# SMALLER DATASET (SUBSET OF FULL) USE THIS FOR QUICK TESTING
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
    recipes5k_food101_test,
) = recipes5k_food101_builder.as_dataset(
    split=["train[:40%]", "train[50%:65%]", "train[95%:]"], shuffle_files=True
)

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(
    nutrition5k_category_train,
    nutrition5k_category_val,
    nutrition5k_category_test,
) = nutrition5k_builder.as_dataset(
    split=["train[:500]", "train[500:650]", "train[950:1000]"], shuffle_files=True
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)
test_dataset = recipes5k_food101_test.concatenate(nutrition5k_category_test)

In [15]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")
print(f"Total testing size : {test_dataset.cardinality().numpy()}")

Total training size : 42830
Total validation size : 16024
Total testing size : 5341


In [16]:
BATCH_SIZE = 32
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
test_dataset = (
    test_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training ingredients and nutrients only

In [ ]:
# FULL DATASET
# no_category_train, no_category_val, no_category_test = nutrition5k_builder.as_dataset(
#     split=["train[:80%]", "train[80%:98%]", "train[98%:]"], shuffle_files=True
# )

In [ ]:
# SMALLER DATASET (SUBSET OF FULL) USE THIS FOR QUICK TESTING
no_category_train, no_category_val, no_category_test = nutrition5k_builder.as_dataset(
    split=["train[:50%]", "train[80%:98%]", "train[98%:]"], shuffle_files=True
)

In [ ]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")
print(f"Total testing size : {no_category_test.cardinality().numpy()}")

In [ ]:
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_test = (
    no_category_test.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

In [17]:
gc.collect()

23

# Model Development

In [18]:
class LRTensorBoard(keras.callbacks.TensorBoard):
    def __init__(self, log_dir, **kwargs):
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs.update({"learning_rate": keras.backend.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)

In [19]:
class BaseModel:
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        self.input_shape = input_shape
        self.input_layer = self.get_input_layer()
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name

    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, previous_layer):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            2048, activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.3)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(self, previous_layer, total_categories):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output"
        )(input_layer)
        return keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )

    def get_shared_nutrition_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        shared_nutrition_regression_layers = keras.layers.Dense(
            256, activation="relu", name="nutrition_dense_1"
        )(input_layer)
        shared_nutrition_regression_layers = keras.layers.BatchNormalization()(
            shared_nutrition_regression_layers
        )
        output_layer = keras.layers.Dropout(0.2)(shared_nutrition_regression_layers)

        return keras.Model(
            inputs=input_layer,
            outputs=output_layer,
            name="nutrition_regression_shared_layers",
        )

    def get_calorie_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        protein_regression_layers = keras.layers.Dense(1, name="protein_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        fat_regression_layers = keras.layers.Dense(1, name="fat_output")(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        raise NotImplementedError

    def _init_layers(self):
        self.augmentation_layers = self.get_augmentation_layers()
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()
        self.shared_layers = self.get_shared_layers(self.convolution_block)

        self.category_classification_layers = self.get_category_classification_layers(
            self.shared_layers, self.total_food_category
        )
        self.shared_nutrition_regression_layers = self.get_shared_nutrition_layers(
            self.shared_layers
        )
        self.calorie_regression_layers = self.get_calorie_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.carbs_regression_layers = self.get_carbs_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.protein_regression_layers = self.get_protein_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.fat_regression_layers = self.get_fat_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.ingredients_multilabel_layers = self.get_ingredients_multilabel_layers(
            self.shared_layers, self.total_ingredients_category
        )

    def _build(self):
        raise NotImplementedError

    def build_and_compile(
        self,
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        category_classification_loss=keras.losses.CategoricalCrossentropy(),
        calorie_regression_loss=keras.losses.MeanAbsoluteError(),
        carbs_regression_loss=keras.losses.MeanAbsoluteError(),
        protein_regression_loss=keras.losses.MeanAbsoluteError(),
        fat_regression_loss=keras.losses.MeanAbsoluteError(),
        ingredient_multilabel_loss=keras.losses.BinaryCrossentropy(),
        category_classification_metrics=[
            keras.metrics.CategoricalAccuracy(),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
            tfa.metrics.F1Score(
                num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                average="macro",
                name="macro_f1",
            ),
            tfa.metrics.F1Score(
                num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                average="micro",
                name="micro_f1",
            ),
        ],
        calorie_regression_metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
        carbs_regression_metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
        protein_regression_metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
        fat_regression_metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
        ingredient_multilabel_metrics=[
            keras.metrics.Precision(),
            keras.metrics.Recall(),
            tfa.metrics.F1Score(
                num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                average="macro",
                name="macro_f1",
            ),
            tfa.metrics.F1Score(
                num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                average="micro",
                name="micro_f1",
            ),
            keras.metrics.TopKCategoricalAccuracy(1, name="Top1_Acc"),
            keras.metrics.TopKCategoricalAccuracy(5, name="Top5_Acc"),
        ],
        category_classification_loss_weights=1.0,
        ingredient_multilabel_loss_weights=1.0,
        calorie_regression_loss_weights=1.0,
        carbs_regression_loss_weights=1.0,
        protein_regression_loss_weights=1.0,
        fat_regression_loss_weights=1.0,
    ):
        model = self._build()
        model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )
        self.model = model
        return model

    def print_summary(self, verbose=True, include_convolution=False):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        if verbose:
            self.augmentation_layers.summary()
            print()
            self.preprocess_layers.summary()
            print()
            if include_convolution:
                self.convolution_block.summary()
                print()
            self.shared_layers.summary()
            print()
            self.category_classification_layers.summary()
            print()
            self.shared_nutrition_regression_layers.summary()
            print()
            self.calorie_regression_layers.summary()
            print()
            self.carbs_regression_layers.summary()
            print()
            self.protein_regression_layers.summary()
            print()
            self.fat_regression_layers.summary()
            print()
            self.ingredients_multilabel_layers.summary()
            print()
        self.model.summary()

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def load_model(self, path):
        self.model = keras.models.load_model(path)
        return self.model

    def train_model(self, enforce_backup_empty=True, **kwargs):
        assert (
            self.model is not None
        ), "No model found. Please call build_and_compile() or load_model() before training."
        backup_dir = Path(f"./temp/backup/{self.name}")
        if enforce_backup_empty and backup_dir.exists():
            assert (
                len(list(backup_dir.iterdir())) == 0
            ), f"WARNING : THERE IS A BACKUP FILE FOR THIS MODEL : {self.name}."

        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def get_callbacks(self):
        tensorboard_dir = f"./models/{self.name}/tensorboard/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
        tensorboard_callback = LRTensorBoard(log_dir=tensorboard_dir)
#         backup_restore_dir = f"../temp/backup/{self.name}"
#         backup_restore_callback = keras.callbacks.BackupAndRestore(
#             backup_dir=backup_restore_dir
#         )
        checkpoint_dir = f"../temp/checkpoint/{self.name}"
        checkpoint_file = checkpoint_dir + "/{epoch:03d}-{val_loss:.4f}.hdf5"
        checkpoint_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_file, monitor="val_loss", save_best_only=True, mode="min"
        )
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=100)
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.1, patience=15, min_lr=1e-6
        )

        return [
            tensorboard_callback,
#             reduce_lr_callback,
#             backup_restore_callback,
            checkpoint_callback,
            early_stopping_callback,
        ]

## Flat Model

In [20]:
class FlatModel(BaseModel):
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        super().__init__(
            input_shape, total_food_category, total_ingredients_category, name
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        ingredients_multilabel_layers = []
        ingredients_multilabel_layers = keras.layers.Dense(
            1024, activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dropout(0.2)(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    def _build(self):
        model_inputs = self.input_layer
        x = self.augmentation_layers(model_inputs)
        x = self.preprocess_layers(x)
        x = self.convolution_block(x)
        x = self.shared_layers(x)
        category_classification_head = self.category_classification_layers(x)
        nutrition_regression_head = self.shared_nutrition_regression_layers(x)
        ingredients_multilabel_head = self.ingredients_multilabel_layers(x)
        calorie_regression_head = self.calorie_regression_layers(
            nutrition_regression_head
        )
        carbs_regression_head = self.carbs_regression_layers(nutrition_regression_head)
        protein_regression_head = self.protein_regression_layers(
            nutrition_regression_head
        )
        fat_regression_head = self.fat_regression_layers(nutrition_regression_head)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

### MobileNetv3

In [21]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_MobileNetv3",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

#### Learning Rate : 0.001 (Tune from here)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.001)
flat_mobilenetv3.print_summary(verbose=True)

In [ ]:
keras.utils.plot_model(
    flat_mobilenetv3.model,
    "./models/flat_foodnet_mobilenetv3.png",
    show_shapes=True,
    expand_nested=False,
)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=25, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : Exponential Decay

In [23]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(
    optimizer=keras.optimizers.Adam(learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        0.1, decay_steps=1339, decay_rate=0.96, staircase=True
    ))
)
flat_mobilenetv3.print_summary(verbose=False)

Model: "FlatFoodNet_with_MobileNetv3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
augmentation_layers (Functional (None, 224, 224, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
preprocessing_layers (Functiona (None, 224, 224, 3)  0           augmentation_layers[0][0]        
__________________________________________________________________________________________________
MobilenetV3large (Functional)   (None, 1, 1, 1280)   4226432     preprocessing_layers[0][0]       
_______________________________________________________________________

In [24]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=25, verbose=1, validation_data=validation_dataset
)

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/25
1335/1339 [============================>.] - ETA: 0s - loss: 17.1570 - category_output_loss: 15.8288 - ingredients_output_loss: 0.1373 - calorie_output_loss: 0.7377 - carbs_output_loss: 0.1761 - protein_output_loss: 0.1206 - fat_output_loss: 0.1564 - category_output_categorical_accuracy: 0.0172 - category_output_precision: 0.0172 - category_output_recall: 0.0172 - category_output_macro_f1: 0.0021 - category_output_micro_f1: 0.0172 - ingredients_output_precision_1: 0.2482 - ingredients_output_recall_1: 0.0618 - ingredients_output_macro_f1: 0.0016 - ingredients_output_micro_f1: 0.1039 - ingredients_output_Top1_Acc: 0.1808 - ingredients_output_Top5_Acc: 0.9249 - calorie_output_MAE: 0.7377 - carbs_output_MAE: 0.1761 - protein_output_MAE: 0.1206 - fat_output_MAE: 0.1564

Cleanup called...


1339/1339 [==============================] - 308s 220ms/step - loss: 17.1573 - category_output_loss: 15.8296 - ingredients_output_loss: 0.1371 - calorie_output_loss: 0.7377 - carbs_output_loss: 0.1759 - protein_output_loss: 0.1206 - fat_output_loss: 0.1563 - category_output_categorical_accuracy: 0.0171 - category_output_precision: 0.0171 - category_output_recall: 0.0171 - category_output_macro_f1: 0.0021 - category_output_micro_f1: 0.0171 - ingredients_output_precision_1: 0.2481 - ingredients_output_recall_1: 0.0617 - ingredients_output_macro_f1: 0.0016 - ingredients_output_micro_f1: 0.1038 - ingredients_output_Top1_Acc: 0.1817 - ingredients_output_Top5_Acc: 0.9251 - calorie_output_MAE: 0.7377 - carbs_output_MAE: 0.1759 - protein_output_MAE: 0.1206 - fat_output_MAE: 0.1563 - val_loss: 17.7957 - val_category_output_loss: 15.8727 - val_ingredients_output_loss: 0.1340 - val_calorie_output_loss: 1.4132 - val_carbs_output_loss: 0.1063 - val_protein_output_loss: 0.1479 - val_fat_output_loss:

Cleanup called...


...

1339/1339 [==============================] - 285s 212ms/step - loss: 16.8709 - category_output_loss: 15.9554 - ingredients_output_loss: 0.1326 - calorie_output_loss: 0.5824 - carbs_output_loss: 0.0911 - protein_output_loss: 0.0352 - fat_output_loss: 0.0741 - category_output_categorical_accuracy: 0.0101 - category_output_precision: 0.0101 - category_output_recall: 0.0101 - category_output_macro_f1: 1.9580e-04 - category_output_micro_f1: 0.0101 - ingredients_output_precision_1: 0.7465 - ingredients_output_recall_1: 0.0161 - ingredients_output_macro_f1: 7.1983e-04 - ingredients_output_micro_f1: 0.0315 - ingredients_output_Top1_Acc: 0.8956 - ingredients_output_Top5_Acc: 1.0000 - calorie_output_MAE: 0.5824 - carbs_output_MAE: 0.0911 - protein_output_MAE: 0.0352 - fat_output_MAE: 0.0741 - val_loss: 67.9386 - val_category_output_loss: 15.9451 - val_ingredients_output_loss: 0.1672 - val_calorie_output_loss: 31.5790 - val_carbs_output_loss: 5.4047 - val_protein_output_loss: 4.9559 - val_fat_out

Cleanup called...


1339/1339 [==============================] - 285s 213ms/step - loss: 16.8711 - category_output_loss: 15.9554 - ingredients_output_loss: 0.1327 - calorie_output_loss: 0.5791 - carbs_output_loss: 0.0915 - protein_output_loss: 0.0378 - fat_output_loss: 0.0746 - category_output_categorical_accuracy: 0.0101 - category_output_precision: 0.0101 - category_output_recall: 0.0101 - category_output_macro_f1: 1.9580e-04 - category_output_micro_f1: 0.0101 - ingredients_output_precision_1: 0.7557 - ingredients_output_recall_1: 0.0146 - ingredients_output_macro_f1: 6.7123e-04 - ingredients_output_micro_f1: 0.0287 - ingredients_output_Top1_Acc: 0.9027 - ingredients_output_Top5_Acc: 1.0000 - calorie_output_MAE: 0.5791 - carbs_output_MAE: 0.0915 - protein_output_MAE: 0.0378 - fat_output_MAE: 0.0746 - val_loss: 113.3981 - val_category_output_loss: 15.9451 - val_ingredients_output_loss: 0.1753 - val_calorie_output_loss: 37.8692 - val_carbs_output_loss: 16.1199 - val_protein_output_loss: 12.5485 - val_fat_

Cleanup called...


1339/1339 [==============================] - 299s 223ms/step - loss: 16.8602 - category_output_loss: 15.9554 - ingredients_output_loss: 0.1326 - calorie_output_loss: 0.5737 - carbs_output_loss: 0.0890 - protein_output_loss: 0.0356 - fat_output_loss: 0.0738 - category_output_categorical_accuracy: 0.0101 - category_output_precision: 0.0101 - category_output_recall: 0.0101 - category_output_macro_f1: 1.9580e-04 - category_output_micro_f1: 0.0101 - ingredients_output_precision_1: 0.7893 - ingredients_output_recall_1: 0.0145 - ingredients_output_macro_f1: 6.7158e-04 - ingredients_output_micro_f1: 0.0285 - ingredients_output_Top1_Acc: 0.8991 - ingredients_output_Top5_Acc: 1.0000 - calorie_output_MAE: 0.5737 - carbs_output_MAE: 0.0890 - protein_output_MAE: 0.0356 - fat_output_MAE: 0.0738 - val_loss: 46.8741 - val_category_output_loss: 15.9451 - val_ingredients_output_loss: 0.1846 - val_calorie_output_loss: 18.1680 - val_carbs_output_loss: 5.0092 - val_protein_output_loss: 3.9694 - val_fat_out

#### Learning Rate : 0.008 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.008)
flat_mobilenetv3.print_summary(verbose=False)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.1 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.1)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.01 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.01)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.004 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.004)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=2, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.01 & Batch Size : 64

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.01)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=20, verbose=1, validation_data=validation_dataset
)

### EfficientNetv2 Model

In [ ]:
class FlatEfficientNetV2Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_EfficientNetV2-Small",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.efficientnet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnetv2S_convolution_layers = (
            keras.applications.efficientnet_v2.EfficientNetV2S(
                input_shape=self.input_shape, include_top=False, weights="imagenet"
            )
        )
        efficientnetv2S_convolution_layers.trainable = False
        return efficientnetv2S_convolution_layers

In [ ]:
flat_efficientnetv2_small = FlatEfficientNetV2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_efficientnetv2_small.build_and_compile()
flat_efficientnetv2_small.print_summary(verbose=False)

In [ ]:
flat_efficientnetv2_small_history = flat_efficientnetv2_small.train_model(
    x=train_dataset, epochs=5, verbose=1, validation_data=validation_dataset
)

### ResNet50 Model (On Hold)

In [ ]:
class FlatResNet50Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_ResNet50",
        )
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.resnet50.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        resnet50_convolution_layers = keras.applications.ResNet50(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        resnet50_convolution_layers.trainable = False
        return resnet50_convolution_layers

In [ ]:
flat_resnet50 = FlatResNet50Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_resnet50.build_and_compile()
flat_resnet50.print_summary()

In [ ]:
flat_resnet50_history = flat_resnet50.train_model(
    train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

## Region Wise Model

In [ ]:
class RegionWiseModel(BaseModel):
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        super().__init__(
            input_shape, total_food_category, total_ingredients_category, name
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        # get the shape of the feature map (batch_size,height,width,channels)
        feature_map_shape = previous_layer.output_shape[1:]
        feature_map_height = feature_map_shape[0]
        feature_map_width = feature_map_shape[1]

        input_layer = keras.layers.Input(feature_map_shape)

        # crop the feature map into grids of feature_map_height * feature_map_width
        region_branches = []
        for row in range(1, feature_map_height + 1):
            top_crop = row - 1
            bottom_crop = feature_map_height - row
            for col in range(1, feature_map_width + 1):
                left_crop = col - 1
                right_crop = feature_map_width - col
                crop_layer = keras.layers.Cropping2D(
                    cropping=((top_crop, bottom_crop), (left_crop, right_crop))
                )(input_layer)
                flatten_layer = keras.layers.Flatten()(crop_layer)
                ingredient_prediction = keras.layers.Dense(
                    total_ingredients, activation="softmax"
                )(flatten_layer)
                reshape_layer = keras.layers.Reshape((1, total_ingredients))(
                    ingredient_prediction
                )  # reshape for global pooling 1D
                region_branches.append(reshape_layer)
        concatenate_layer = keras.layers.Concatenate(axis=1)(region_branches)
        global_max_pooling_layer = keras.layers.GlobalMaxPool1D()(concatenate_layer)
        return keras.Model(
            input_layer, global_max_pooling_layer, name="ingredients_output"
        )

    def _build(self):
        model_inputs = self.input_layer
        x = self.augmentation_layers(model_inputs)
        x = self.preprocess_layers(x)
        x = self.convolution_block(x)
        ingredients_multilabel_head = self.ingredients_multilabel_layers(x)
        x = self.shared_layers(x)
        category_classification_head = self.category_classification_layers(x)
        nutrition_regression_head = self.shared_nutrition_regression_layers(x)
        calorie_regression_head = self.calorie_regression_layers(
            nutrition_regression_head
        )
        carbs_regression_head = self.carbs_regression_layers(nutrition_regression_head)
        protein_regression_head = self.protein_regression_layers(
            nutrition_regression_head
        )
        fat_regression_head = self.fat_regression_layers(nutrition_regression_head)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

### MobileNetv3 Model

In [ ]:
class RegionWiseMobileNetv3Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_MobileNetv3",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

In [ ]:
regionwise_mobilenetv3 = RegionWiseMobileNetv3Model(
    (224, 224, 3),
    len(data_pipeline.one_hot_encoder.all_food_categories),
    len(data_pipeline.one_hot_encoder.all_ingredients),
)
regionwise_mobilenetv3.build_and_compile()
regionwise_mobilenetv3.print_summary()

In [ ]:
keras.utils.plot_model(
    regionwise_mobilenetv3.model,
    "./models/regionwise_foodnet_mobilenetv3.png",
    show_shapes=True,
    expand_nested=False,
)

In [ ]:
regionwise_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

### EfficientNetv2 Model

In [ ]:
class RegionWiseEfficientNetV2Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_EfficientNetv2-Small",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.efficientnet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnetv2S_convolution_layers = (
            keras.applications.efficientnet_v2.EfficientNetV2S(
                input_shape=self.input_shape, include_top=False, weights="imagenet"
            )
        )
        efficientnetv2S_convolution_layers.trainable = False
        return efficientnetv2S_convolution_layers

In [ ]:
regionwise_efficientnetv2_small = RegionWiseEfficientNetV2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
regionwise_efficientnetv2_small.build_and_compile()
regionwise_efficientnetv2_small.print_summary(verbose=False)

In [ ]:
regionwise_efficientnetv2_small_history = regionwise_efficientnetv2_small.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

# Testing

In [ ]:
for x in test:
    print(x)

In [ ]:
tf.keras.utils.array_to_img(x[0].numpy())

In [25]:
import os
os.chdir(r'/kaggle/working')

!tar -czf models.tar.gz ./models

from IPython.display import FileLink

FileLink(r'models.tar.gz')

/kaggle/working/models.tar.gz

In [1]:
import shutil
shutil.rmtree("./models/")